In [2]:
import json
from time import time
import pandas as pd
from pathlib import Path
from dpmm.pipelines import MSTPipeline

/tmp/ipykernel_5503/1878173709.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Load data

In [3]:
wine_dir = Path().parent / "wine"

df = pd.read_pickle(wine_dir / "wine.pkl.gz")
with (wine_dir / "wine_bounds.json").open("r") as f:
    domain = json.load(f)

df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4192,white,6.3,0.21,0.29,11.7,0.048,49.0,147.0,0.99482,3.22,0.38,10.8,0
2157,white,5.9,0.21,0.24,12.1,0.044,53.0,165.0,0.99690,3.25,0.39,9.5,0
631,red,10.4,0.28,0.54,2.7,0.105,5.0,19.0,0.99880,3.25,0.63,9.5,0
3410,white,7.6,0.38,0.20,3.4,0.046,9.0,116.0,0.99440,3.15,0.41,9.4,0
3117,white,8.4,0.23,0.49,7.8,0.035,22.0,95.0,0.99350,3.04,0.34,12.0,1


# Build pipeline

In [ ]:
pipeline = MSTPipeline(
    epsilon=1,  # Privacy budget for generate model
    proc_epsilon=0.1,  # Privacy budget for data processing 
    delta=1e-5,  # Delta Setting in (eps,  delta) differential privacy
)

# Fit model

In [5]:
print("Fitting MST pipeline...", end='\r')
start_time = time()
# The number of rows to generate
pipeline.fit(df, domain)
fit_time = time()
print(f"Fitting MST pipeline - Took {fit_time - start_time:.2f} seconds")

Fitting MST pipeline - Took 16.08 seconds


# Conditional generation

In [13]:
print("Generating synthetic data - with provided", end="\r")

# Generate synthetic data
condition_records = df.iloc[:, :3]
synth_df = pipeline.generate(condition_records=condition_records, random_state=2025)
gen_time = time()

print(f"Generating synthetic data - Took {gen_time - fit_time:.2f} seconds")

print("-" * 30, "CONDITION RECORDS", "-" * 30)
display(condition_records.head())

print("-" * 30, "SYNHTETIC DATA", "-" * 30)
display(synth_df.head())


Generating synthetic data - Took 0.07 seconds
------------------------------ CONDITION RECORDS ------------------------------


,type,fixed acidity,volatile acidity
4192,white,6.3,0.21
2157,white,5.9,0.21
631,red,10.4,0.28
3410,white,7.6,0.38
3117,white,8.4,0.23


------------------------------ SYNHTETIC DATA ------------------------------


,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4192,white,6.3,0.21,0.276249,11.239081,0.051467,27.421273,129.516896,0.996181,3.202751,0.238570,10.685489,1
2157,white,5.9,0.21,0.368127,4.239620,0.041612,12.271191,94.717443,0.990143,3.301406,0.568494,12.101755,1
631,red,10.4,0.28,0.356594,8.381831,0.073808,26.384701,172.392042,0.997316,3.271858,0.627237,9.310492,0
3410,white,7.6,0.38,0.696183,6.839125,0.239474,4.568832,13.049544,0.997468,3.399397,0.455332,10.943835,0
3117,white,8.4,0.23,0.775516,10.482141,0.045293,44.199538,138.762952,0.997003,3.202850,0.398027,9.964178,0
